In [ ]:
renv::load(here::here())
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(magrittr)
library(padr)
library(kableExtra)
library(purrr)
library(ggplot2)
library(ggthemr)
library(here)
library(lubridate)
library(AeRobiology)

ggthemr("fresh")
devtools::load_all()

load(paste0(here(), "/data/other/species.RData"))
load(paste0(here(), "/data/other/stations.RData"))
species %<>%
  mutate(fieldextra_taxon = str_replace_all(fieldextra_taxon, "1", "24"))
data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_daily.txt"))


In [ ]:
pollen_split <- data_dwh %>%
  filter(date >= as.Date("2010-01-01"),
         date <= as.Date("2020-12-31")) %>%
  mutate(year = year(date),
         tuning_factor_orig = case_when(
           taxon == "Alnus" ~ 0.34,
           taxon == "Betula" ~ 0.63,
           taxon == "Poaceae" ~ 0.26
         )) %>%
  select(taxon, station, year, datetime, value, tuning_factor_orig) %>%
  split(.$taxon) %>%
    map(~ .x %>% split(list(.$year, .$station)))

In [ ]:

season_def_alnu <- map(pollen_split$Alnus, ~.x %>%
  select(datetime, value) %>%
  calculate_ps(
    method = "clinical",
    n.clinical = 5,
    window.clinical = 7,
    th.pollen = 10,
    th.sum = 100,
    plot = FALSE,
    result = "table"
  ))

season_def_betu <- map(pollen_split$Betula, ~.x %>%
  select(datetime, value) %>%
  calculate_ps(
    method = "clinical",
    n.clinical = 5,
    window.clinical = 7,
    th.pollen = 10,
    th.sum = 100,
    plot = FALSE,
    result = "table"
  ))

season_def_poac <- map(pollen_split$Poaceae, ~.x %>%
  select(datetime, value) %>%
  calculate_ps(
    method = "clinical",
    n.clinical = 5,
    window.clinical = 7,
    th.pollen = 3,
    th.sum = 30,
    plot = FALSE,
    result = "table"
  ))

season_def <- c(season_def_alnu, season_def_betu, season_def_poac)

season_start <- map(season_def, ~ .x$st.jd) %>% unlist
season_end <- map(season_def, ~ .x$en.jd) %>% unlist

pollen_clean <- unlist(pollen_split, recursive = FALSE)
pollen_clean <- pollen_clean[-which(is.na(season_start))]
season_start <- season_start[!is.na(season_start)]
season_end <- season_end[!is.na(season_end)]

pollen_in_season <- pmap(list(pollen_clean, season_start, season_end), ~
..1 %>%
  slice(..2:..3)) %>%
  bind_rows()


In [39]:
pollen_in_season %>%
  group_by(taxon, station, tuning_factor_orig) %>%
  mutate(average_season = sum(value, na.rm = TRUE) / length(unique(pollen_in_season$year))) %>%
  group_by(taxon, station, year, tuning_factor_orig, average_season) %>%
  summarise(current_season = sum(value, na.rm = TRUE)) %>%
  ungroup() %>%
  mutate(tuning_factor_current = tuning_factor_orig / average_season * current_season) %>%
  group_by(taxon) %>%
  summarise(max = max(tuning_factor_current),
  min = min(tuning_factor_current))


`summarise()` has grouped output by 'taxon', 'station', 'year', 'tuning_factor_orig'. You can override using the `.groups` argument.



,taxon,max,min
,<chr>,<dbl>,<dbl>
1,Alnus,0.9602157,0.08875944
2,Betula,2.1239028,0.15532126
3,Poaceae,0.4061470,0.11698138
